In [1]:
import requests
import json
import sqlite3
from sqlite3 import Error
import pandas as pd

In [ ]:
headers = {
    'x-rapidapi-key': "38bab05df5msh928211553580d9cp181186jsne583ad5bd5cd",
    'x-rapidapi-host': "free-nba.p.rapidapi.com"
    }
page = 1
results = []
while True:


    url = "https://free-nba.p.rapidapi.com/stats"

    querystring = {"page": page,"per_page":"100"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    data = response.json()
    if len(data['data']) <= 0:
        break
    results += data['data']
    print(response.json())
    break

    page += 1

    print(response.json())



In [ ]:
with open("data.json", "w") as data_file:
    json.dump(results, data_file, indent=4)
    data_file.close()

In [ ]:
print(data["data"][0]['fg3_pct'])

In [6]:

# found how to create db in sqlite3 here:  https://www.sqlitetutorial.net/sqlite-python/insert/

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        c = conn.cursor()
        print(sqlite3.version)
        # this is extra code to create table and load csv into table.  Was testing here
        c.execute('''CREATE TABLE players (PLAYER_NAME text, TEAM_ID text, PLAYER_ID text, SEASON int)''')
        players = pd.read_csv('players.csv')
        players.to_sql('players', conn, if_exists='append', index=False)
        c.execute('''SELECT * FROM players''').fetchall()

    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_connection(r"nba_test.db")

2.6.0
table players already exists


In [9]:
# another method of creating db and tables but without function: https://mungingdata.com/sqlite/create-database-load-csv-python/
import sqlite3
# create connection to build new db if nonexistent and cursor to execute queries
conn = sqlite3.connect('nba_test')
c = conn.cursor()

In [10]:
# create table players **testing to see if works**
c.execute('''CREATE TABLE players (PLAYER_NAME text, TEAM_ID text, PLAYER_ID text, SEASON int)''')

In [12]:
# using pandas to read csv then push data to table created in last cell
players = pd.read_csv('players.csv')
players.to_sql('players', conn, if_exists='append', index=False)


In [13]:
# querying table to verify creation
c.execute('''SELECT * FROM players LIMIT 5''').fetchall()

[("Royce O'Neale", '1610612762', '1626220', 2019),
 ('Bojan Bogdanovic', '1610612762', '202711', 2019),
 ('Rudy Gobert', '1610612762', '203497', 2019),
 ('Donovan Mitchell', '1610612762', '1628378', 2019),
 ('Mike Conley', '1610612762', '201144', 2019)]

In [14]:
# create rest of tables
teams = pd.read_csv('teams.csv')
teams.to_sql('teams', conn, if_exists='append', index=False)
games = pd.read_csv('games.csv')
games.to_sql('games', conn, if_exists='append', index=False)
games_details = pd.read_csv('games_details.csv')
games_details.to_sql('games_details', conn, if_exists='append', index=False)

In [21]:
# Kobe ID is 977, Lebron ID is 2544
# testing select statement for filtering by player id
c.execute('''SELECT * FROM players WHERE PLAYER_ID = 2544''').fetchall()

[('LeBron James', '1610612747', '2544', 2019),
 ('LeBron James', '1610612747', '2544', 2018),
 ('LeBron James', '1610612739', '2544', 2017),
 ('LeBron James', '1610612739', '2544', 2016),
 ('LeBron James', '1610612739', '2544', 2015),
 ('LeBron James', '1610612739', '2544', 2014),
 ('LeBron James', '1610612748', '2544', 2013),
 ('LeBron James', '1610612748', '2544', 2012),
 ('LeBron James', '1610612748', '2544', 2011),
 ('LeBron James', '1610612748', '2544', 2010),
 ('LeBron James', '1610612739', '2544', 2009),
 ('LeBron James', '1610612747', '2544', 2019),
 ('LeBron James', '1610612747', '2544', 2018),
 ('LeBron James', '1610612739', '2544', 2017),
 ('LeBron James', '1610612739', '2544', 2016),
 ('LeBron James', '1610612739', '2544', 2015),
 ('LeBron James', '1610612739', '2544', 2014),
 ('LeBron James', '1610612748', '2544', 2013),
 ('LeBron James', '1610612748', '2544', 2012),
 ('LeBron James', '1610612748', '2544', 2011),
 ('LeBron James', '1610612748', '2544', 2010),
 ('LeBron Jam

In [22]:
# testing select statement to query additional stats and filtering by player id
c.execute('''SELECT TEAM_ABBREVIATION, PLAYER_NAME, FG_PCT, FG3_PCT, FT_PCT, PTS FROM games_details WHERE PLAYER_ID = 2544''').fetchall()


.423, 0.0, 0.833, 27.0),
 ('CLE', 'LeBron James', 0.3, 0.0, 0.8, 16.0),
 ('CLE', 'LeBron James', 0.368, 0.0, 0.7140000000000001, 19.0),
 ('CLE', 'LeBron James', 0.435, 0.0, 0.5, 21.0),
 ('CLE', 'LeBron James', 0.353, 0.0, 1.0, 14.0),
 ('CLE', 'LeBron James', 0.316, 0.0, 0.5, 14.0),
 ('CLE', 'LeBron James', 0.45, 0.0, 0.667, 22.0),
 ('CLE', 'LeBron James', 0.652, 0.0, 1.0, 32.0),
 ('CLE', 'LeBron James', 0.292, 0.25, 0.75, 18.0),
 ('CLE', 'LeBron James', 0.464, 0.4, 0.8, 34.0),
 ('CLE', 'LeBron James', 0.318, 0.16699999999999998, 0.875, 22.0),
 ('CLE', 'LeBron James', 0.5, 0.0, 0.833, 32.0),
 ('CLE', 'LeBron James', 0.583, 0.7140000000000001, 1.0, 36.0),
 ('CLE', 'LeBron James', 0.368, 0.5, 0.5, 17.0),
 ('CLE', 'LeBron James', 0.625, 0.0, 0.875, 27.0),
 ('CLE', 'LeBron James', 0.5, 0.2, 0.889, 37.0),
 ('CLE', 'LeBron James', 0.381, 0.333, 0.75, 23.0),
 ('CLE', 'LeBron James', 0.438, 0.0, 1.0, 18.0),
 ('CLE', 'LeBron James', 0.333, 0.0, 0.0, 8.0),
 ('CLE', 'LeBron James', 0.154, 0.0, 0.0

In [23]:
# testing select statement to query additional stats and filtering by player id in a DataFrame
pd.read_sql('''SELECT TEAM_ABBREVIATION, PLAYER_NAME, FG_PCT, FG3_PCT, FT_PCT, PTS FROM games_details WHERE PLAYER_ID = 2544''', conn)


,TEAM_ABBREVIATION,PLAYER_NAME,FG_PCT,FG3_PCT,FT_PCT,PTS
0,LAL,LeBron James,0.500,0.500,1.000,10.0
1,LAL,LeBron James,0.609,0.444,0.833,37.0
2,LAL,LeBron James,0.571,0.571,1.000,25.0
3,LAL,LeBron James,0.615,0.500,0.667,22.0
4,LAL,LeBron James,0.364,0.429,1.000,26.0
...,...,...,...,...,...,...
1684,MIA,LeBron James,0.250,0.250,0.833,10.0
1685,MIA,LeBron James,0.545,0.000,0.500,13.0
1686,MIA,LeBron James,0.625,0.000,0.500,11.0
1687,MIA,LeBron James,0.545,0.000,0.889,20.0
